# Tarefa 03

- Leia os enunciados com atenção
- Saiba que pode haver mais de uma resposta correta
- Insira novas células de código sempre que achar necessário
- Em caso de dúvidas, procure os Tutores
- Divirta-se :)

In [1]:
import pandas as pd
import requests # type: ignore

####  1) Lendo de APIs
Vimos em aula como carregar dados públicos do governo através de um API (*Application Programming Interface*). No exemplo de aula, baixamos os dados de pedidos de verificação de limites (PVL) realizados por estados, e selecionamos apenas aqueles referentes ao estado de São Paulo.

1. Repita os mesmos passos feitos em aula, mas selecione os PVLs realizados por municípios no estado do Rio de Janeiro.
2. Quais são os três *status* das solicitações mais frequentes na base? Quais são suas frequências?
3. Construa uma nova variável que contenha o ano do **status**. Observe que ```data_status``` vem como tipo *object* no **DataFrame**. Dica: você pode usar o método ```.str``` para transformar o tipo da variável em string, em seguida um método como [**slice()**](https://pandas.pydata.org/docs/reference/api/pandas.Series.str.slice.html) ou [**split()**](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.str.split.html).
4. Indique a frequência de cada ano do campo construído no item (3).

In [2]:
# 1) Seu código aqui

url = 'https://apidatalake.tesouro.gov.br/ords/sadipem/tt/pvl?uf=RJ&tipo_interessado=Munic%C3%ADpio'

r = requests.get(url)

r.status_code


200

In [3]:
# 2) Seu código aqui

df = pd.DataFrame(r.json()['items'])
df['status'].value_counts().nlargest(3)

status
Deferido        106
Arquivado        53
Regularizado     26
Name: count, dtype: int64

In [4]:
df.head()

,id_pleito,tipo_interessado,interessado,cod_ibge,uf,num_pvl,status,num_processo,data_protocolo,tipo_operacao,finalidade,tipo_credor,credor,moeda,valor,pvl_assoc_divida,pvl_contradado_credor,data_status
0,32212,Município,Miguel Pereira,3302908,RJ,PVL02.001363/2019-61,Deferido (PVL-IF),None,None,Operação contratual interna,"Aquisição de máquinas, equipamentos e veículos",Instituição Financeira Nacional,Agência de Fomento do Estado do Rio de Janeiro,Real,4500000.0,0,1,22/07/2019
1,34096,Município,Resende,3304201,RJ,PVL02.003087/2019-75,Deferido (PVL-IF),None,None,Operação contratual interna,Aporte de capital,Instituição Financeira Nacional,Caixa Econômica Federal,Real,4950000.0,0,1,27/09/2019
2,62991,Município,Paraíba do Sul,3303708,RJ,PVL02.000023/2023-07,Deferido (PVL-IF),None,None,Operação contratual interna,"Aquisição de máquinas, equipamentos e veículos",Instituição Financeira Nacional,Banco do Brasil S/A,Real,1100000.0,0,1,28/03/2023
3,62992,Município,Paraíba do Sul,3303708,RJ,PVL02.000024/2023-43,Deferido (PVL-IF),None,None,Operação contratual interna,Infraestrutura,Instituição Financeira Nacional,Banco do Brasil S/A,Real,4299482.4,0,1,28/03/2023
4,7216,Município,São João de Meriti,3305109,RJ,None,Arquivado a pedido,17944.000523/2010-89,2012-05-02T03:00:00Z,Operação contratual interna,Caminho da Escola 2009 A,Instituição Financeira Nacional,Banco do Brasil S/A,Real,1745900.0,0,0,21/05/2012


In [5]:
# 3 e 4) Seu código aqui
df['data_status'] = pd.to_datetime(df['data_status'], format='%d/%m/%Y')
df['ano_status'] = df['data_status'].dt.year
df['ano_status'].value_counts()

ano_status
2008    37
2007    29
2023    28
2014    20
2010    19
2019    16
2013    15
2020    15
2012    13
2022    11
2011    10
2009     9
2015     8
2017     5
2021     5
2018     4
2006     4
2016     4
2002     4
2024     3
2003     1
2004     1
Name: count, dtype: int64

####  2) Melhorando a interação com o API
Observe dois URLs de consultas diferentes, por exemplo o URL utilizado em aula, e o URL feito no exercício anterior. Compare-os e observe as diferenças.

1. Faça uma função em Python que recebe como argumento o UF da consulta e o tipo de interessado (```'Estado'```ou ```Município```), e que devolve os dados da consulta no formato *DataFrame*.
2. Quantas solicitações para o Estado podem ser consultadas para Minas Gerais com *status* em 'Arquivado por decurso de prazo' estão registradas?
3. Qual é o município da Bahia com mais solicitações deferidas?
4. Salve um arquivo .csv com os dados de solicitações da Bahia, com interessado = 'Estado'

In [6]:
url_sp = 'https://apidatalake.tesouro.gov.br/ords/sadipem/tt/pvl?uf=SP&tipo_interessado=Estado'
url_rj = 'https://apidatalake.tesouro.gov.br/ords/sadipem/tt/pvl?uf=RJ&tipo_interessado=Munic%C3%ADpio'

In [7]:
#1) Seu código aqui

def get_pvl_data(uf, tipo_interessado):
    url = f'https://apidatalake.tesouro.gov.br/ords/sadipem/tt/pvl?uf={uf}&tipo_interessado={tipo_interessado}'
    r = requests.get(url)
    df = pd.DataFrame(r.json()['items'])
    return df

In [8]:
# 2) Seu código aqui

df_pvl_mg_estado = get_pvl_data('MG', 'Estado')
df_pvl_mg_estado[df_pvl_mg_estado['status'] == 'Arquivado por decurso de prazo'].shape[0]

1

In [9]:
# 3) Seu código aqui

df_pvl_ba_municipio = get_pvl_data('BA', 'Município')
deferimento = (df_pvl_ba_municipio['status'] == 'Deferido') | (df_pvl_ba_municipio['status'] == 'Deferido (PVL-IF)')
df_pvl_ba_municipio_deferido = df_pvl_ba_municipio[deferimento]
df_pvl_ba_municipio_deferido['interessado'].value_counts().nlargest(1)

interessado
Luís Eduardo Magalhães    16
Name: count, dtype: int64

In [10]:
df_pvl_ba_municipio.head()

,id_pleito,tipo_interessado,interessado,cod_ibge,uf,num_pvl,status,num_processo,data_protocolo,tipo_operacao,finalidade,tipo_credor,credor,moeda,valor,pvl_assoc_divida,pvl_contradado_credor,data_status
0,32153,Município,Cairu,2905404,BA,PVL02.001315/2019-72,Deferido (PVL-IF),None,None,Operação contratual interna,Infraestrutura,Instituição Financeira Nacional,Agência de Fomento do Estado da Bahia S/A,Real,4480000.0,0,0,22/05/2019
1,31372,Município,Santa Brígida,2927606,BA,PVL02.000521/2019-65,Em retificação pelo interessado,17944.101204/2019-28,2019-04-05T14:09:11Z,Operação contratual interna,Regularização de Dívida - Energia Elétrica,Empresa Não Estatal,Companhia de Eletricidade do Estado da Bahia,Real,48721.3,0,0,11/04/2019
2,61562,Município,Simões Filho,2930709,BA,PVL02.005914/2022-61,Deferido,17944.103664/2022-96,2022-11-01T15:58:20Z,Operação contratual interna,Infraestrutura,Instituição Financeira Nacional,Banco do Brasil S/A,Real,11500000.0,0,1,04/11/2022
3,65975,Município,João Dourado,2918357,BA,PVL02.002964/2023-77,Deferido (PVL-IF),None,None,Operação contratual interna,Infraestrutura,Instituição Financeira Nacional,Caixa Econômica Federal,Real,10000000.0,0,0,22/09/2023
4,67131,Município,Lauro de Freitas,2919207,BA,PVL02.003848/2023-75,Deferido,17944.105270/2023-53,2023-11-13T15:20:47Z,Operação contratual interna,Infraestrutura,Instituição Financeira Nacional,Banco do Brasil S/A,Real,25000000.0,0,1,17/11/2023


In [11]:
# 4) Seu código aqui

df_pvl_ba_estado = get_pvl_data('BA', 'Estado')
df_pvl_ba_estado.to_csv('PVL Estadual Bahia.csv')